# Amazon Bedrock Introduction - Nova Models
This notebook demonstrates how to use Amazon Bedrock with Nova models

In [1]:
# %% Import necessary packages
import boto3
import json
from pprint import pprint

In [2]:
# %% Initialize Bedrock clients
bedrock = boto3.client('bedrock', region_name='us-east-1')
bedrock_runtime = boto3.client('bedrock-runtime', region_name='us-east-1')

In [3]:
# %% List all available model IDs
response = bedrock.list_foundation_models()

print("All Available Models:")
print("=" * 80)
for model in response['modelSummaries']:
    print(f"Model ID: {model['modelId']}")
    print(f"Provider: {model['providerName']}")
    print(f"Model Name: {model['modelName']}")
    print("-" * 80)

All Available Models:
Model ID: nvidia.nemotron-nano-12b-v2
Provider: NVIDIA
Model Name: NVIDIA Nemotron Nano 12B v2 VL BF16
--------------------------------------------------------------------------------
Model ID: anthropic.claude-sonnet-4-20250514-v1:0
Provider: Anthropic
Model Name: Claude Sonnet 4
--------------------------------------------------------------------------------
Model ID: anthropic.claude-haiku-4-5-20251001-v1:0
Provider: Anthropic
Model Name: Claude Haiku 4.5
--------------------------------------------------------------------------------
Model ID: openai.gpt-oss-120b-1:0
Provider: OpenAI
Model Name: gpt-oss-120b
--------------------------------------------------------------------------------
Model ID: stability.stable-creative-upscale-v1:0
Provider: Stability AI
Model Name: Stable Image Creative Upscale
--------------------------------------------------------------------------------
Model ID: qwen.qwen3-next-80b-a3b
Provider: Qwen
Model Name: Qwen3 Next 80B A3B
--

In [4]:
# %% List only Nova model IDs and ARNs
response = bedrock.list_foundation_models()

print("Amazon Nova Models:")
print("=" * 80)
nova_models = []
for model in response['modelSummaries']:
    if 'nova' in model['modelId'].lower():
        nova_models.append(model)
        print(f"Model ID: {model['modelId']}")
        print(f"Model ARN: {model['modelArn']}")
        print(f"Model Name: {model['modelName']}")
        print(f"Input Modalities: {model.get('inputModalities', [])}")
        print(f"Output Modalities: {model.get('outputModalities', [])}")
        print("-" * 80)

print(f"\nTotal Nova models found: {len(nova_models)}")

Amazon Nova Models:
Model ID: amazon.nova-2-multimodal-embeddings-v1:0
Model ARN: arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-2-multimodal-embeddings-v1:0
Model Name: Amazon Nova Multimodal Embeddings
Input Modalities: ['TEXT', 'IMAGE', 'AUDIO', 'VIDEO']
Output Modalities: ['EMBEDDING']
--------------------------------------------------------------------------------
Model ID: amazon.nova-pro-v1:0
Model ARN: arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-pro-v1:0
Model Name: Nova Pro
Input Modalities: ['TEXT', 'IMAGE', 'VIDEO']
Output Modalities: ['TEXT']
--------------------------------------------------------------------------------
Model ID: amazon.nova-2-lite-v1:0
Model ARN: arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-2-lite-v1:0
Model Name: Nova 2 Lite
Input Modalities: ['TEXT', 'IMAGE', 'VIDEO']
Output Modalities: ['TEXT']
--------------------------------------------------------------------------------
Model ID: amazon.nova-2-lite-v1:0:256k
Model A

In [5]:
# %% Make an invoke_model request with Nova Lite v1
model_id = "amazon.nova-lite-v1:0"

prompt = "What are the key features of heavy machinery used in construction?"

request_body = {
    "messages": [
        {
            "role": "user",
            "content": [
                {"text": prompt}
            ]
        }
    ],
    "inferenceConfig": {
        "max_new_tokens": 500,
        "temperature": 0.7,
        "top_p": 0.9
    }
}

response = bedrock_runtime.invoke_model(
    modelId=model_id,
    contentType='application/json',
    accept='application/json',
    body=json.dumps(request_body)
)

response_body = json.loads(response['body'].read())

print("invoke_model Response:")
print("=" * 80)
print(f"Model: {model_id}")
print(f"Prompt: {prompt}")
print("\nResponse:")
print(response_body['output']['message']['content'][0]['text'])
print("\nMetadata:")
pprint(response_body.get('usage', {}))

invoke_model Response:
Model: amazon.nova-lite-v1:0
Prompt: What are the key features of heavy machinery used in construction?

Response:
Heavy machinery used in construction is designed to handle large, demanding tasks that require significant power and durability. Here are some key features of these machines:

### 1. **Power and Performance**
- **High Horsepower Engines:** Typically diesel engines, providing substantial power for heavy lifting and moving.
- **Torque:** High torque capabilities to manage heavy loads and tough terrain.

### 2. **Durability**
- **Robust Construction:** Built to withstand harsh working conditions and frequent use.
- **Materials:** Often made from high-strength steel and other durable materials to enhance longevity.

### 3. **Versatility**
- **Attachments:** Various attachments (e.g., buckets, forks, blades) that can be swapped out for different tasks.
- **Multi-functionality:** Designed to perform multiple tasks, such as excavation, lifting, and grading.

In [6]:
# %% Make a converse request with Nova Lite v1
model_id = "amazon.nova-lite-v1:0"

conversation = [
    {
        "role": "user",
        "content": [
            {"text": "What is the typical operating weight of a large bulldozer?"}
        ]
    }
]

response = bedrock_runtime.converse(
    modelId=model_id,
    messages=conversation,
    inferenceConfig={
        "maxTokens": 500,
        "temperature": 0.7,
        "topP": 0.9
    }
)

print("Converse API Response:")
print("=" * 80)
print(f"Model: {model_id}")
print(f"User: {conversation[0]['content'][0]['text']}")
print("\nAssistant Response:")
print(response['output']['message']['content'][0]['text'])
print("\nUsage:")
pprint(response['usage'])
print(f"\nStop Reason: {response['stopReason']}")

Converse API Response:
Model: amazon.nova-lite-v1:0
User: What is the typical operating weight of a large bulldozer?

Assistant Response:
The typical operating weight of a large bulldozer can vary significantly depending on the specific model and manufacturer. However, most large bulldozers generally fall within a range of approximately 40,000 to 100,000 pounds (about 18,000 to 45,000 kilograms). 

For example:
- A Caterpillar D11T bulldozer typically has an operating weight of around 80,000 to 90,000 pounds (36,000 to 41,000 kilograms).
- A Komatsu D355A bulldozer generally weighs around 60,000 to 70,000 pounds (27,000 to 32,000 kilograms) in operating condition.

These weights can vary based on the configuration, attachments, and specific model features. Always refer to the manufacturer's specifications for the most accurate information.

Usage:
{'inputTokens': 11, 'outputTokens': 199, 'totalTokens': 210}

Stop Reason: end_turn


In [10]:
# %% Make a converse request with Nova Pro v1 for image analysis
import base64
from pathlib import Path

model_id = "us.amazon.nova-pro-v1:0"

# Option 1: Load image from file
# image_path = "path/to/your/image.jpg"
# with open(image_path, "rb") as image_file:
#     image_bytes = image_file.read()

# Option 2: Use a sample base64 encoded image (1x1 red pixel for demo)
# Replace this with actual image bytes
sample_image = base64.b64decode(
    "iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAYAAAAfFcSJAAAADUlEQVR42mP8z8DwHwAFBQIAX8jx0gAAAABJRU5ErkJggg=="
)

conversation = [
    {
        "role": "user",
        "content": [
            {
                "image": {
                    "format": "png",
                    "source": {
                        "bytes": sample_image
                    }
                }
            },
            {
                "text": "Analyze this image and describe what type of heavy machinery or equipment you see. Include details about its condition, purpose, and any safety features visible."
            }
        ]
    }
]

response = bedrock_runtime.converse(
    modelId=model_id,
    messages=conversation,
    inferenceConfig={
        "maxTokens": 1000,
        "temperature": 0.5,
        "topP": 0.9
    }
)

print("Image Analysis with Nova Pro:")
print("=" * 80)
print(f"Model: {model_id}")
print("\nAnalysis Result:")
print(response['output']['message']['content'][0]['text'])
print("\nUsage:")
pprint(response['usage'])
print(f"\nStop Reason: {response['stopReason']}")

print("\n" + "=" * 80)
print("NOTE: Replace 'sample_image' with actual heavy machinery image bytes")
print("Supported formats: PNG, JPEG, GIF, WebP")
print("Max image size: 3.75 MB (base64 encoded)")

Image Analysis with Nova Pro:
Model: us.amazon.nova-pro-v1:0

Analysis Result:
The image shows a large, red piece of heavy machinery, likely a construction vehicle such as a bulldozer or an excavator. The machinery is predominantly red, with a solid, robust appearance. It appears to be in good condition, with no visible damage or wear. The vehicle is equipped with large, sturdy wheels designed for rough terrain, indicating its use in construction or heavy-duty tasks. Safety features include a protective cage around the operator's compartment, suggesting it is designed to protect the operator in case of accidents. Additionally, there are visible warning labels and reflective markings, enhancing visibility and safety on the job site.

Usage:
{'inputTokens': 561, 'outputTokens': 123, 'totalTokens': 684}

Stop Reason: end_turn

NOTE: Replace 'sample_image' with actual heavy machinery image bytes
Supported formats: PNG, JPEG, GIF, WebP
Max image size: 3.75 MB (base64 encoded)


In [11]:
# %% Multi-turn conversation example with Nova Lite
model_id = "us.amazon.nova-lite-v1:0"

# Start a conversation
messages = [
    {
        "role": "user",
        "content": [{"text": "What is a bulldozer used for?"}]
    }
]

# First turn
response = bedrock_runtime.converse(
    modelId=model_id,
    messages=messages,
    inferenceConfig={"maxTokens": 300, "temperature": 0.7}
)

print("Multi-turn Conversation:")
print("=" * 80)
print(f"User: {messages[0]['content'][0]['text']}")
print(f"\nAssistant: {response['output']['message']['content'][0]['text']}")

# Add assistant response to conversation
messages.append(response['output']['message'])

# Second turn
messages.append({
    "role": "user",
    "content": [{"text": "What are the main safety considerations when operating one?"}]
})

response = bedrock_runtime.converse(
    modelId=model_id,
    messages=messages,
    inferenceConfig={"maxTokens": 300, "temperature": 0.7}
)

print("\n" + "-" * 80)
print(f"User: {messages[2]['content'][0]['text']}")
print(f"\nAssistant: {response['output']['message']['content'][0]['text']}")
print("\n" + "=" * 80)
print(f"Total tokens used: {response['usage']['totalTokens']}")

Multi-turn Conversation:
User: What is a bulldozer used for?

Assistant: A bulldozer is a heavy equipment machine commonly used in construction, demolition, and land-clearing projects. Its primary function is to move large amounts of soil, debris, and other materials. Here are some of the main uses of a bulldozer:

1. **Earthmoving**: Bulldozers are used to push and level large quantities of soil, gravel, or other materials. They can create flat surfaces for building foundations, roads, or other structures.

2. **Clearing Land**: In construction and land development, bulldozers are used to clear vegetation, trees, and other obstacles from a site. This is often done to prepare land for construction or agriculture.

3. **Snow Removal**: In winter, bulldozers can be used to clear snow from roads, parking lots, and other areas to ensure safe passage and accessibility.

4. **Grading**: Bulldozers help in grading the land to achieve the desired slope and elevation for various purposes, such 